In [1]:
import numpy as np
from SALib.sample import saltelli
from SALib.analyze import sobol

# hymod as defined by Gharari et al. HESS 2013
# load daily data for 1 year (P,PET,Q)
ndays = 365
data = np.loadtxt('leaf-river-data.txt', skiprows=1)
data_P = data[0:ndays,3]
data_PET = data[0:ndays,4]
data_Q = data[0:ndays,5]

def hymod(x, mode='optimize'):

  # assign parameters
  Sm_max,B,alpha,Kf,Ks = list(x)

  # initialize storage, all empty to start
  Sm,Sf1,Sf2,Sf3,Ss1 = [np.zeros(ndays) for _ in range(5)]
  Q = np.zeros(ndays)

  for t in range(1,ndays):

    # calculate all fluxes
    P = data_P[t]
    Peff = P*(1 - max(1-Sm[t-1]/Sm_max,0)**B) # PDM model Moore 1985
    Evap = min(data_PET[t]*(Sm[t-1]/Sm_max), Sm[t-1])

    Qf1 = Kf*Sf1[t-1]
    Qf2 = Kf*Sf2[t-1]
    Qf3 = Kf*Sf3[t-1]
    Qs1 = Ks*Ss1[t-1]
    
    # update state variables
    Sm[t] = Sm[t-1] + P - Peff - Evap
    Sf1[t] = Sf1[t-1] + alpha*Peff - Qf1
    Sf2[t] = Sf2[t-1] + Qf1 - Qf2
    Sf3[t] = Sf3[t-1] + Qf2 - Qf3
    Ss1[t] = Ss1[t-1] + (1-alpha)*Peff - Qs1

    Q[t] = Qs1 + Qf3

  if mode=='simulate':
    return Q
  else:
    return np.sqrt(np.mean((Q-data_Q)**2))


problem = {
  'num_vars': 5,
  'names': ['Cmax', 'B', 'alpha', 'Kq', 'Ks'],
  'bounds': [(0,500), (0,2), (0,1), (0.1,1), (0,0.1)]
}

param_values = saltelli.sample(problem, 1000)
N = len(param_values) # number of parameter samples
Y = np.zeros(N)

# Run model for each parameter set, save the output in array Y
for i in range(N):
  if i % 1000 == 0:
    print(i)
  Y[i] = hymod(param_values[i])

Si = sobol.analyze(problem, Y, print_to_console=True)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
Parameter S1 S1_conf ST ST_conf
Cmax 0.101145 0.040323 0.169630 0.030668
B 0.072793 0.035554 0.111594 0.026993
alpha 0.311073 0.059318 0.584154 0.126089
Kq 0.209991 0.065921 0.484375 0.085439
Ks 0.004130 0.012163 0.010668 0.002799

Parameter_1 Parameter_2 S2 S2_conf
Cmax B -0.027794 0.063706
Cmax alpha -0.000625 0.083645
Cmax Kq 0.009421 0.084902
Cmax Ks -0.026197 0.057630
B alpha -0.007812 0.060822
B Kq -0.004674 0.053906
B Ks -0.017494 0.043834
alpha Kq 0.190793 0.144138
alpha Ks 0.004986 0.072984
Kq Ks -0.014286 0.099882
